In [3]:
from xclim import ensembles
from pathlib import Path
import xarray as xr
from CMIP6_IO import CMIP6_IO
import os
from dask.distributed import Client
import dask
import gcsfs

## Create ensemble files from running CMIP6_light.py

This script will loop over all files found in a speficic folder `lightpath` and find files of the same variable and scenario to create ensemble files from. The various variables to create ensembles for include `["uvb", "uv", "uvi", "par", "uva", "tos", "siconc", "sithick", "tas", "sisnthick", "sisnconc", "uas", "vas"]`.

The output is stored under folder `ensemble_path` and the results are used to create the final plots of modeled lightlyfor the paper using notebooks `CMIP6_plot_light_results.ipynb` and `CMIP6_calculate_MPIESM2.ipynb`.

This script may require substantial RAM depending on the region you are creating ensemble for. For datasets covering the Northern Hemisphere we used a 64CPU and 120GB RAM machine.


In [4]:
# Define what models, scenarios, and variables to calculate ensemble files from

source_ids = ["MPI-ESM1-2-HR","MPI-ESM1-2-LR", "CanESM5", "UKESM1-0-LL"]

# Weights file is created using CMIP6_create_weights.ipynb together with 
# the ClimWIP package. The weights are calculated for variable tos using ERSSTv5 and WOA18 as observations
# to compare with. The weights are calculated for the period 1993-2020.
weights_file = "data/Calculated_weights_26092023.nc"

create_ensemble = True
scenarios = ["ssp245", "ssp585"]
var_names = ["par_", "uvb_", "uv_", "uvi_", "uva_", "uv_srf_"]  # ,"_tos.nc","_siconc.nc"]
var_names = ["par", "tos", "uvb", "uv"] #, "uvi", "uva"] #, "_tos.nc","_siconc.nc"]
var_names = ["tos", "uvb", "uv", "uva"]
var_names = ["sithick", "tas", "sisnthick", "sisnconc", "uas", "vas"]
var_names = ["tos", "uvb", "uv", "uva", "siconc", "sithick", "tas", "sisnthick", "sisnconc", "uas", "vas"]
var_names = ["par", "uvb", "uv", "uva"]

percentiles = [2.5, 50.0, 97.5]

create_ensemble_files = True
io = CMIP6_IO()

In [5]:
import numpy as np

def get_weights(weights_file: str, f: str):
    weights = xr.open_dataset(weights_file)

    weights = weights["weights"]
    models = weights["model_ensemble"]
    for source_id in source_ids:
        if source_id in f:
            for d, w in zip(models.values, weights.values):
                if source_id in d.split("_")[0]:
                    print(f"Found model in models {source_id} here {d} weight {w}")
                    return w
 
def create_ensemble_files(scenarios, var_names):
    max_models=300
    only_perc= True
    for use_weights in [True, False]:
        
        for var_name in var_names:
            for scenario in scenarios:
                counter=0
                ds_list = []
                ensemble_stats = None
                ensemble_perc = None
                if var_name in ["prw", "clt", "tos", "siconc", "sithick", "tas", "sisnthick", "sisnconc", "uas", "vas"]:
                    lightpath = "light"
                    ensemble_path = "light/ensemble"
                else:
                    lightpath = f"light/ncfiles/{scenario}"
                    ensemble_path = f"light/ncfiles/{scenario}/ensemble"
                if use_weights:
                    ensemble_stats =  f"{ensemble_path}/CMIP6_ensemble_stats_{scenario}_{var_name}_weighted.nc"
                    ensemble_perc =  f"{ensemble_path}/CMIP6_ensemble_perc_{scenario}_{var_name}_weighted.nc"
                else:
                    ensemble_stats =  f"{ensemble_path}/CMIP6_ensemble_stats_{scenario}_{var_name}.nc"
                    ensemble_perc =  f"{ensemble_path}/CMIP6_ensemble_perc_{scenario}_{var_name}.nc"
                 
                if not os.path.exists(ensemble_path):
                    os.makedirs(ensemble_path, exist_ok=True)

                assert (
                    ensemble_stats is not None
                ), "Unable to identify correct variable name to create output filename"
                assert (
                    ensemble_perc is not None
                ), "Unable to identify correct variable name to create output filename"

                if os.path.exists(ensemble_stats):
                    os.remove(ensemble_stats)
                if os.path.exists(ensemble_perc):
                    os.remove(ensemble_perc)

                current = f"{lightpath}"
                print(current)
                file_on_gcs = io.list_dataset_on_gs(current)
                
                # Loop over all files and filter on the models defined in source_ids.
                # For each model read the corresponding weight from the weights file. We
                # will use these values to create a weights xr.Dataarray to be used to weight each 
                # model when calculating the ensemble mean, std, max, min, and percentiles.
                models_weights = []
                for f in file_on_gcs:
                    
                    var_name_mod = var_name
                    if var_name not in ["prw", "clt", "tos", "siconc", "sithick", "tas", "sisnthick", "sisnconc", "uas", "vas"]:
                        var_name_mod = f"{var_name}_"
                        
                    # Filter to only use the models we prefer
                    if any(model in f for model in source_ids) and var_name_mod in f and "uv_srf" not in f and scenario in f:
                        if counter >= max_models:
                            pass
                        else:
                            if "MPI-ESM1-2-HR_r2i1p1f1" not in f:
                                ds = io.open_dataset_on_gs(f)
                                print("=======")
                                print(scenario, f)
                                
                                models_weights.append(get_weights(weights_file, f))
                                
                                # Drop variable mask as we dont need it and it causes problems when
                                # calculating the ensemble mean.
                                if "mask" in ds.variables:
                                    ds = ds.drop("mask")
                                
                                ds_list.append(ds)
                            
                                counter+=1
                # Create the final xr.DataArray with the weights.    
                d = {
                    "dims": "realization",
                    "data": np.array(models_weights),
                    "name": "weights",
                }
                print(f"Number of datasets found {len(ds_list)}")
                weights = xr.DataArray.from_dict(d)
              
                print(f"Creating ensemble for {var_name} and scenario {scenario}")
                if var_name in ["tos"]:
                    ens = ensembles.create_ensemble(ds_list, resample_freq="MS")
                else:
                    ens = ensembles.create_ensemble(ds_list)
              
                if use_weights:
                    if only_perc is False:
                        ens_stats = ensembles.ensemble_mean_std_max_min(ens, weights=weights)
                    else:
                        ens_perc = ensembles.ensemble_percentiles(
                          ens, values=percentiles, split=False #, weights=weights
                      )   
                else:
                    if only_perc is False:
                        ens_stats = ensembles.ensemble_mean_std_max_min(ens)
                    else:
                        ens_perc = ensembles.ensemble_percentiles(
                          ens, values=percentiles, split=False
                      )

                # Save to file and upload to GCS.
                if only_perc is False:
                    ens_stats.to_netcdf(ensemble_stats)
                    io.upload_to_gcs(ensemble_stats)
                    print(f"Created ensemble {ensemble_stats}")
                else:
                    ens_perc.to_netcdf(ensemble_perc)
                    io.upload_to_gcs(ensemble_perc)
                    print(f"Created ensemble {ensemble_perc}")
                
                if Path(ensemble_stats).exists():
                    Path(ensemble_stats).unlink()
                if Path(ensemble_perc).exists():
                     Path(ensemble_perc).unlink()
                

dask.config.set(**{"array.slicing.split_large_chunks": True})  
with Client() as client: # set up local cluster on your laptop
    create_ensemble_files(scenarios, var_names)

/home/sam/miniconda3/envs/actea-3.9/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39489 instead
  warnings.warn(


light/ncfiles/ssp245
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
ssp245 light/ncfiles/ssp245/par_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
ssp245 light/ncfiles/ssp245/par_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
ssp245 light/ncfiles/ssp245/par_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/par_Can

/home/tk/.local/lib/python3.9/site-packages/xclim/core/utils.py:524: RuntimeWarning: All-NaN slice encountered
  result = np.where(np.isnan(interpolation), np.nanmax(arr, axis=0), interpolation)


Created ensemble light/ncfiles/ssp245/ensemble/CMIP6_ensemble_perc_ssp245_par_weighted.nc
light/ncfiles/ssp585
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp585/par_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
ssp585 light/ncfiles/ssp585/par_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp585/par_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
ssp585 light/ncfiles/ssp585/par_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp585/par_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
ssp585 light/ncfiles/ssp585/par_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 wei

/home/tk/.local/lib/python3.9/site-packages/xclim/core/utils.py:524: RuntimeWarning: All-NaN slice encountered
  result = np.where(np.isnan(interpolation), np.nanmax(arr, axis=0), interpolation)


...

Created ensemble light/ncfiles/ssp585/ensemble/CMIP6_ensemble_perc_ssp585_uv.nc
light/ncfiles/ssp245
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/uva_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
ssp245 light/ncfiles/ssp245/uva_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/uva_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
ssp245 light/ncfiles/ssp245/uva_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp245/uva_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
ssp245 light/ncfiles/ssp245/uva_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp245.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.0172

/home/tk/.local/lib/python3.9/site-packages/xclim/core/utils.py:524: RuntimeWarning: All-NaN slice encountered
  result = np.where(np.isnan(interpolation), np.nanmax(arr, axis=0), interpolation)


Created ensemble light/ncfiles/ssp245/ensemble/CMIP6_ensemble_perc_ssp245_uva.nc
light/ncfiles/ssp585
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp585/uva_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
ssp585 light/ncfiles/ssp585/uva_CanESM5_r10i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp585/uva_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
ssp585 light/ncfiles/ssp585/uva_CanESM5_r1i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017222982059866998
[CMIP6_IO] Opening file actea-shared/light/ncfiles/ssp585/uva_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
ssp585 light/ncfiles/ssp585/uva_CanESM5_r2i1p2f1_1979-01-01-2099-12-16_scenario_osa_ssp585.nc
Found model in models CanESM5 here CanESM5_r1i1p1f1_CMIP6 weight 0.017

/home/tk/.local/lib/python3.9/site-packages/xclim/core/utils.py:524: RuntimeWarning: All-NaN slice encountered
  result = np.where(np.isnan(interpolation), np.nanmax(arr, axis=0), interpolation)


Created ensemble light/ncfiles/ssp585/ensemble/CMIP6_ensemble_perc_ssp585_uva.nc
